In [1]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.inception_v3 import preprocess_input
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
# import matplotlib.pyplot as plt
from keras.applications.resnet import ResNet50
import tensorflow as tf
from tensorflow.keras.applications.xception import decode_predictions

# from tf_explain.core.activations import ExtractActivations

In [2]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]

train_path = '/home/aamruth/Documents/BACK UP/test zenith/CNN'
#valid_path = '/home/vimalkumar/Documents/softco/CNN/CNN/test'

In [3]:
InceptionV3=InceptionV3(input_shape=(224,224,3), weights='imagenet', include_top=False)

2022-07-02 15:13:23.140281: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-07-02 15:13:23.140332: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-07-02 15:13:23.140353: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (admininistrator): /proc/driver/nvidia/version does not exist
2022-07-02 15:13:23.140587: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
# don't train existing weights
for layer in InceptionV3.layers:
    layer.trainable = False

In [5]:
folders = glob('/home/aamruth/Documents/BACK UP/test zenith/CNN/*')
len(folders)

78

In [6]:
x = Flatten()(InceptionV3.output)

In [7]:
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=InceptionV3.input, outputs=prediction)

In [8]:
# tell the model what cost and optimization method to use
model.compile(
  loss='binary_crossentropy',
  optimizer='rmsprop',
  metrics=['accuracy']
)

In [9]:
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255,
#                                    rotation_range=20,
#                                    zoom_range=0.15,
#                                    width_shift_range=0.2,
#                                    height_shift_range=0.2,
#                                    shear_range=0.15,
#                                    horizontal_flip=True,
#                                    fill_mode="nearest",
                                   validation_split = 0.15)
#test_datagen = ImageDataGenerator(rescale = 1./255)
#test_datagen = ImageDataGenerator(rescale = 1./255)

In [10]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (224, 224),
                                                 batch_size = 4,
                                                 class_mode = 'categorical',
                                                subset = 'training')

test_set = train_datagen.flow_from_directory(train_path,
                                            target_size = (224, 224),
                                            batch_size = 4,
                                            class_mode = 'categorical',
                                           subset = 'validation')

Found 41716 images belonging to 78 classes.
Found 7327 images belonging to 78 classes.


In [11]:
training_set.class_indices

{'A&P RETAIL': 0,
 'AUDIT': 1,
 'Albertons': 2,
 'Amagamated meat cutter': 3,
 'BOENKER STOP N STOP': 4,
 'DAYTON MEAT LOCAL 1099': 5,
 "Domimck's": 6,
 'FISHER FOODS': 7,
 'FOOD GIANT': 8,
 'FOOD HANDLERS': 9,
 'FURRS INC': 10,
 'Fisher Food of Canton': 11,
 'GROCERY COMPANY': 12,
 'HEALTH WELFARE & PCR': 13,
 'Kroger': 14,
 'Kroger pension contrib': 15,
 'LOCAL 1099': 16,
 'LOCAL 305': 17,
 'LOCAL 408': 18,
 'LOCAL 442': 19,
 'LOCAL 610': 20,
 'LOCAL 76 PENSION REPORT': 21,
 'LOCAL 99': 22,
 'LOCAL UNION BENEFIT': 23,
 'Local 1059': 24,
 'MAGNETIC TAPE PROCESSING': 25,
 'MC HEALTH WELFARE': 26,
 'MONTH OF': 27,
 'MTHLY Pension Report': 28,
 'NATIONAL TEA': 29,
 'Northside Packing': 30,
 'OHIO Store': 31,
 'PANTRY': 32,
 'PATRICK CUDAHY': 33,
 'PENSION REPORT 2': 34,
 'PITTSBURGH MEAT': 35,
 'POULTRY PROCESSING': 36,
 'Patrick': 37,
 'Pension Contribution Report': 38,
 'Pension Contributions': 39,
 'Pension Plan Report': 40,
 'Pick and Pay Type 1': 41,
 'Pick and Pay Type 2': 42,
 'Pi

In [12]:
r = model.fit(
  training_set,
  validation_data=test_set,
  epochs=5,
  #steps_per_epoch=len(training_set),
  #validation_steps=len(test_set)
)

Epoch 1/5
10429/10429 [==============================] - 2784s 267ms/step - loss: 0.0634 - accuracy: 0.8823 - val_loss: 0.0820 - val_accuracy: 0.8923
Epoch 2/5
10429/10429 [==============================] - 2722s 261ms/step - loss: 0.0295 - accuracy: 0.9574 - val_loss: 0.0769 - val_accuracy: 0.9001
Epoch 3/5
10429/10429 [==============================] - 2704s 259ms/step - loss: 0.0207 - accuracy: 0.9729 - val_loss: 0.0818 - val_accuracy: 0.8990
Epoch 4/5
10429/10429 [==============================] - 2703s 259ms/step - loss: 0.0155 - accuracy: 0.9806 - val_loss: 0.0883 - val_accuracy: 0.8912
Epoch 5/5
10429/10429 [==============================] - 2797s 268ms/step - loss: 0.0125 - accuracy: 0.9841 - val_loss: 0.0765 - val_accuracy: 0.9116


In [13]:
model.save('zenith_cnn(1991).h5')

In [1]:
from tensorflow import keras

model = keras.models.load_model('zenith_cnn(1991).h5')

In [2]:
import os
from tensorflow.keras.utils import load_img,img_to_array
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
import numpy as np
import pandas as pd
from PIL import Image
import pytesseract as pt
import cv2
from PIL import Image 
import PIL 
import os.path

In [ ]:
p='D:\\CNN\\Unclassified'
dic = { 0:'A&P RETAIL',
 1:'AUDIT',
 2:'Albertons',
 3:'Amagamated meat cutter',
 4:'BOENKER STOP N STOP',
 5:'DAYTON MEAT LOCAL 1099',
 6:"Domimck's",
 7:'FISHER FOODS',
 8:'FOOD GIANT',
 9:'FOOD HANDLERS',
 10:'FURRS INC',
 11:'Fisher Food of Canton',
 12:'GROCERY COMPANY',
 13:'HEALTH WELFARE & PCR',
 14:'Kroger',
 15:'Kroger pension contrib',
 16:'LOCAL 1099',
 17:'LOCAL 305',
 18:'LOCAL 408',
 19:'LOCAL 442',
 20:'LOCAL 610',
 21:'LOCAL 76 PENSION REPORT',
 22:'LOCAL 99',
 23:'LOCAL UNION BENEFIT',
 24:'Local 1059',
 25:'MAGNETIC TAPE PROCESSING',
 26:'MC HEALTH WELFARE',
 27:'MONTH OF',
 28:'MTHLY Pension Report',
 29:'NATIONAL TEA',
 30:'Northside Packing',
 31:'OHIO Store',
 32:'PANTRY',
 33:'PATRICK CUDAHY',
 34:'PENSION REPORT 2',
 35:'PITTSBURGH MEAT',
 36:'POULTRY PROCESSING',
 37:'Patrick',
 38:'Pension Contribution Report',
 39:'Pension Contributions',
 40:'Pension Plan Report',
 41:'Pick and Pay Type 1',
 42:'Pick and Pay Type 2',
 43:'Pick and Pay Type 3',
 44:'RINI Supermarket',
 45:'Riserfoods',
 46:'SAFEWAY STORES',
 47:'SAFEWAY STORES INC',
 48:'SANDY MAC',
 49:'SAVON MEAT LOCAL',
 50:'SHOP AND STOP',
 51:'STOLL FOOD',
 52:'SUPER PLUS',
 53:'Seaway Food',
 54:'Smith',
 55:'Smith Management Corp',
 56:'Super Valu Store',
 57:'TAMARKIN COMPANY',
 58:'THE GRAND',
 59:'Termination Report',
 60:'The Great Atlantic',
 61:'U.F.C.W',
 62:'UFCW NATIONAL',
 63:'UFCW National Pension Fund',
 64:'UN NAMED',
 65:'UNION 424',
 66:'UNION HOURS',
 67:'UNITED FOOD',
 68:'UNITED FOOD AND COMMERCIAL',
 69:'UNITED STORES',
 70:'UPHW SYSTEM',
 71:'Unclassified',
 72:'joseph',
 73:'local union no',
 74:'monthly union hour',
 75:'newhire report',
 76:'rego barera',
 77:'supermarket general corporation'}
file_name = []
doc_type = []
cnt = 1
for im in os.listdir(p):
    im_name=im.split('.')[0]
    print(im_name) 
    cnt += 1
    img_1=cv2.imread(os.path.join(p,im))
#     resize=cv2.resize(img,(224,224))
    img = image.load_img(os.path.join(p,im), 
                     target_size=(224,224))
    file_name.append(im)
# #     plt.imshow(img)
# #     plt.show()
#     dictnary=dic[a[0]]
    x = image.img_to_array(img)
    x = x/255
    x = np.expand_dims(x,axis=0)
    images = np.vstack([x])
    val = model.predict(images)
    
    
    a=np.argmax(val,axis=1)
    print(cnt)
    print(dic[a[0]])
    doc_type.append(dic[a[0]])
df = pd.DataFrame({'File':file_name,'Result':doc_type})
# pd.set_option('display.max_rows',1000)
# display(df)

O100025-1
1/1 [==============================] - 0s 120ms/step
2
Unclassified
O100025-2
1/1 [==============================] - 0s 112ms/step
3
Unclassified
O100025-20
1/1 [==============================] - 0s 120ms/step
4
Unclassified
O100025-21
1/1 [==============================] - 0s 120ms/step
5
Unclassified
O100025-22
1/1 [==============================] - 0s 120ms/step
6
Unclassified
O100025-3
1/1 [==============================] - 0s 112ms/step
7
Unclassified
O100066-1
1/1 [==============================] - 0s 120ms/step
8
Unclassified
O100066-2
1/1 [==============================] - 0s 112ms/step
9
Unclassified
O100066-21
1/1 [==============================] - 0s 112ms/step
10
Unclassified
O100080-1
1/1 [==============================] - 0s 120ms/step
11
Unclassified
O100080-11
1/1 [==============================] - 0s 112ms/step
12
Unclassified
O100080-2
1/1 [==============================] - 0s 112ms/step
13
Unclassified
O100080-20
1/1 [==============================] - 0s 13

1/1 [==============================] - 0s 136ms/step
105
Unclassified
O48865-2
1/1 [==============================] - 0s 136ms/step
106
Unclassified
O48865-8
1/1 [==============================] - 0s 136ms/step
107
Unclassified
O48872-18
1/1 [==============================] - 0s 128ms/step
108
Unclassified
O48920-1
1/1 [==============================] - 0s 136ms/step
109
Unclassified
O48920-2
1/1 [==============================] - 0s 136ms/step
110
Unclassified
O48920-4
1/1 [==============================] - 0s 136ms/step
111
Unclassified
O48920-5
1/1 [==============================] - 0s 128ms/step
112
Unclassified
O48920-6
1/1 [==============================] - 0s 128ms/step
113
Unclassified
O48928-1
1/1 [==============================] - 0s 128ms/step
114
Unclassified
O48928-2
1/1 [==============================] - 0s 128ms/step
115
Unclassified
O49152-22
1/1 [==============================] - 0s 144ms/step
116
Unclassified
O49180-11
1/1 [==============================] - 0s 136ms/s

1/1 [==============================] - 0s 136ms/step
208
Unclassified
O51634-3
1/1 [==============================] - 0s 136ms/step
209
Unclassified
O51634-5
1/1 [==============================] - 0s 136ms/step
210
Unclassified
O51655-1
1/1 [==============================] - 0s 136ms/step
211
Unclassified
O51655-12
1/1 [==============================] - 0s 128ms/step
212
Unclassified
O51655-15
1/1 [==============================] - 0s 128ms/step
213
Unclassified
O51655-3
1/1 [==============================] - 0s 136ms/step
214
Unclassified
O51655-6
1/1 [==============================] - 0s 136ms/step
215
Unclassified
O51655-9
1/1 [==============================] - 0s 136ms/step
216
Unclassified
O51701-1
1/1 [==============================] - 0s 128ms/step
217
Unclassified
O51701-2
1/1 [==============================] - 0s 128ms/step
218
Unclassified
O51701-23
1/1 [==============================] - 0s 136ms/step
219
Unclassified
O51701-3
1/1 [==============================] - 0s 128ms/s

1/1 [==============================] - 0s 152ms/step
311
Unclassified
O53769-12
1/1 [==============================] - 0s 144ms/step
312
Unclassified
O53769-14
1/1 [==============================] - 0s 136ms/step
313
Unclassified
O53769-2
1/1 [==============================] - 0s 152ms/step
314
Unclassified
O53769-4
1/1 [==============================] - 0s 152ms/step
315
Unclassified
O53769-6
1/1 [==============================] - 0s 144ms/step
316
Unclassified
O53769-8
1/1 [==============================] - 0s 144ms/step
317
Unclassified
O53788-41
1/1 [==============================] - 0s 144ms/step
318
Unclassified
O53790-1
1/1 [==============================] - 0s 144ms/step
319
Unclassified
O53790-23
1/1 [==============================] - 0s 144ms/step
320
Unclassified
O53790-26
1/1 [==============================] - 0s 136ms/step
321
Unclassified
O53790-38
1/1 [==============================] - 0s 136ms/step
322
Unclassified
O53790-44
1/1 [==============================] - 0s 136

1/1 [==============================] - 0s 136ms/step
414
UNITED FOOD
O56296-1
1/1 [==============================] - 0s 128ms/step
415
Unclassified
O56296-10
1/1 [==============================] - 0s 128ms/step
416
Unclassified
O56296-26
1/1 [==============================] - 0s 144ms/step
417
MAGNETIC TAPE PROCESSING
O56296-27
1/1 [==============================] - 0s 136ms/step
418
Unclassified
O56297-1
1/1 [==============================] - 0s 128ms/step
419
Unclassified
O56297-29
1/1 [==============================] - 0s 128ms/step
420
UNITED FOOD
O56351-1
1/1 [==============================] - 0s 128ms/step
421
Unclassified
O56351-2
1/1 [==============================] - 0s 128ms/step
422
Unclassified
O56352-1
1/1 [==============================] - 0s 128ms/step
423
Unclassified
O56353-1
1/1 [==============================] - 0s 136ms/step
424
Unclassified
O56354-1
1/1 [==============================] - 0s 136ms/step
425
Unclassified
O56355-1
1/1 [==============================] -

...

1/1 [==============================] - 0s 136ms/step
2868
Unclassified
O78864-16
1/1 [==============================] - 0s 144ms/step
2869
Unclassified
O78864-17
1/1 [==============================] - 0s 136ms/step
2870
Unclassified
O78864-2
1/1 [==============================] - 0s 136ms/step
2871
Unclassified
O78864-23
1/1 [==============================] - 0s 136ms/step
2872
Unclassified
O78864-4
1/1 [==============================] - 0s 136ms/step
2873
Unclassified
O78864-5
1/1 [==============================] - 0s 136ms/step
2874
Unclassified
O78864-6
1/1 [==============================] - 0s 136ms/step
2875
Unclassified
O78864-7
1/1 [==============================] - 0s 144ms/step
2876
Unclassified
O78864-8
1/1 [==============================] - 0s 136ms/step
2877
Unclassified
O78864-9
1/1 [==============================] - 0s 136ms/step
2878
Unclassified
O78885-15
1/1 [==============================] - 0s 144ms/step
2879
Unclassified
O78892-17
1/1 [==============================

1/1 [==============================] - 0s 136ms/step
2970
Unclassified
O79585-12
1/1 [==============================] - 0s 136ms/step
2971
Unclassified
O79585-16
1/1 [==============================] - 0s 144ms/step
2972
Unclassified
O79585-2
1/1 [==============================] - 0s 136ms/step
2973
Unclassified
O79585-20
1/1 [==============================] - 0s 144ms/step
2974
Unclassified
O79588-1
1/1 [==============================] - 0s 136ms/step
2975
Unclassified
O79588-13
1/1 [==============================] - 0s 136ms/step
2976
Unclassified
O79588-15
1/1 [==============================] - 0s 144ms/step
2977
Unclassified
O79588-2
1/1 [==============================] - 0s 136ms/step
2978
Unclassified
O79591-1
1/1 [==============================] - 0s 136ms/step
2979
Unclassified
O79591-6
1/1 [==============================] - 0s 136ms/step
2980
Unclassified
O79592-44
1/1 [==============================] - 0s 136ms/step
2981
TAMARKIN COMPANY
O79673-1
1/1 [=========================

1/1 [==============================] - 0s 136ms/step
3072
Unclassified
O80433-1
1/1 [==============================] - 0s 136ms/step
3073
Unclassified
O80433-17
1/1 [==============================] - 0s 144ms/step
3074
Unclassified
O80433-2
1/1 [==============================] - 0s 136ms/step
3075
Unclassified
O80434-1
1/1 [==============================] - 0s 144ms/step
3076
Unclassified
O80434-2
1/1 [==============================] - 0s 136ms/step
3077
Unclassified
O80434-21
1/1 [==============================] - 0s 136ms/step
3078
Unclassified
O80435-1
1/1 [==============================] - 0s 144ms/step
3079
Unclassified
O80435-5
1/1 [==============================] - 0s 136ms/step
3080
Unclassified
O80472-52
1/1 [==============================] - 0s 136ms/step
3081
Unclassified
O80473-1
1/1 [==============================] - 0s 136ms/step
3082
Unclassified
O80473-3
1/1 [==============================] - 0s 136ms/step
3083
Unclassified
O80473-5
1/1 [==============================] 

1/1 [==============================] - 0s 136ms/step
3174
Unclassified
O81570-2
1/1 [==============================] - 0s 136ms/step
3175
Unclassified
O81570-21
1/1 [==============================] - 0s 136ms/step
3176
Unclassified
O81570-23
1/1 [==============================] - 0s 136ms/step
3177
UNITED FOOD
O81570-25
1/1 [==============================] - 0s 136ms/step
3178
Unclassified
O81570-26
1/1 [==============================] - 0s 128ms/step
3179
UNITED FOOD
O81570-27
1/1 [==============================] - 0s 152ms/step
3180
UNITED FOOD
O81570-28
1/1 [==============================] - 0s 136ms/step
3181
UNITED FOOD
O81570-29
1/1 [==============================] - 0s 136ms/step
3182
UNITED FOOD
O81570-3
1/1 [==============================] - 0s 157ms/step
3183
Unclassified
O81570-4
1/1 [==============================] - 0s 136ms/step
3184
Unclassified
O81570-6
1/1 [==============================] - 0s 136ms/step
3185
Unclassified
O81580-1
1/1 [==============================] -

1/1 [==============================] - 0s 144ms/step
3276
Unclassified
O82010-1
1/1 [==============================] - 0s 144ms/step
3277
Unclassified
O82010-2
1/1 [==============================] - 0s 128ms/step
3278
Unclassified
O82010-20
1/1 [==============================] - 0s 144ms/step
3279
Unclassified
O82010-25
1/1 [==============================] - 0s 136ms/step
3280
Unclassified
O82010-30
1/1 [==============================] - 0s 136ms/step
3281
Unclassified
O82010-35
1/1 [==============================] - 0s 136ms/step
3282
Unclassified
O82010-40
1/1 [==============================] - 0s 136ms/step
3283
Unclassified
O82010-7
1/1 [==============================] - 0s 136ms/step
3284
Unclassified
O82011-1
1/1 [==============================] - 0s 152ms/step
3285
Unclassified
O82011-10
1/1 [==============================] - 0s 136ms/step
3286
Unclassified
O82011-19
1/1 [==============================] - 0s 136ms/step
3287
Unclassified
O82024-1
1/1 [============================

In [ ]:
df['Result'].value_counts()